# Traversability Simulation
## Stucture

```
.
├── agent # Agent package
├── data
├── docker
│   ├── Dockerfile
│   ├── Dockerfile-roscore
│   ├── Dockerfile-webots
│   └── nvidia-docker.sh
├── docker-compose.yml
├── filename2map.csv
├── gym.ipynb
├── main.py
├── maps
├── notebooks # useful notebooks
├── parser.py
├── protocols # shared Protocols
├── __pycache__
│   └── parser.cpython-36.pyc
├── README.md
├── requirements.txt
├── simulation # Simulation package
├── start.sh
├── test.py
├── utils
│   ├── History.py
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── History.cpython-36.pyc
│   │   └── __init__.cpython-36.pyc
│   └── webots2ros
│       ├── __init__.py
│       ├── __pycache__
│       ├── README.md
│       ├── Supervisor.py
│       └── test.py
├── webots
│   ├── __init__.py
│   ├── krock # Agent and Env for krock + webots
│   ├── map_generation
│   ├── utils.py
│   └── WebotsWorld.py
└── world # World package
```

## Webots + Krock
![alt text](https://raw.githubusercontent.com/FrancescoSaverioZuppichini/Master-Thesis/gym/resources/images/krock.jpg")

In order to run the Krock robot using Webots we provide a [gym](https://gym.openai.com/) env. If you know gym, then you are good to do. It follows a **basic example** (API could change)

In [19]:
import rospy
import pprint

from webots.krock import Krock, KrockWebotsEnv
from webots import WebotsWorld

from simulation.conditions import *

WORLD_PATH = '/home/francesco/Documents/Master-Thesis/core/webots/krock/krock2_ros/worlds/holes2.wbt'
EPISODES = 25

rospy.init_node("traversability_simulation")

world = WebotsWorld.from_file(WORLD_PATH)
krock = Krock()
# call world and krock to initialise them
world()
krock()
# at this point under the hood the world file was imported, loader and all the ros subs and pubs are initialized
env = KrockWebotsEnv.from_file(WORLD_PATH, should_stop=IfOneFalseOf([IsInside(), IsNotStuck(n_last=50)]))
# spawn the robot at a random location
init_obs = env.reset()

print('Initial observations:')
pprint.pprint(init_obs)

for _ in range(EPISODES):
#   go forward for a while
    obs, r, done, _ = env.step(env.GO_FORWARD)
    pprint.pprint(obs)
    if done: break
        
env.step(env.STOP)

/home/francesco/Documents/Master-Thesis/core/webots/krock/krock2_ros/worlds/holes2.wbt


[WARN] [1542396677.743687]: /krock/supervisor/get_self transport error completing service call: unable to receive data from sender, check sender's logs for details


Initial observations:
{'sensors': {'front_cam': None,
             'orientation': {'w': -0.41029170833578366,
                             'x': 0.5913878435814885,
                             'y': 0.44728050649657597,
                             'z': 0.6709698703907674},
             'position': {'x': 5.006386148428594,
                          'y': -4.619462922813117,
                          'z': 1.0746239881091906}}}
{'sensors': {'front_cam': None,
             'orientation': {'w': 0.9940487797760903,
                             'x': -0.009098581458505336,
                             'y': 0.9999526176725474,
                             'z': -0.0034609572754965134},
             'position': {'x': 5.0040842186708385,
                          'y': -4.619508880238725,
                          'z': 1.1116198026671276}}}
{'sensors': {'front_cam': None,
             'orientation': {'w': 1.0574748998589119,
                             'x': -0.006729483837493061,
                  

{'sensors': {'front_cam': None,
             'orientation': {'w': 0.5213441845953157,
                             'x': -0.3057361234848099,
                             'y': 0.9181735476115229,
                             'z': -0.2519578521557753},
             'position': {'x': 5.190662566730865,
                          'y': -4.87714722009238,
                          'z': 0.5547716082690978}}}
{'sensors': {'front_cam': None,
             'orientation': {'w': 0.5682263374432178,
                             'x': -0.30761295259384136,
                             'y': 0.9220995169249036,
                             'z': -0.2347482742926104},
             'position': {'x': 5.201012197472762,
                          'y': -4.894953651364779,
                          'z': 0.5539834415777914}}}
{'sensors': {'front_cam': None,
             'orientation': {'w': 0.6202631644104558,
                             'x': -0.2229840413602415,
                             'y': 0.9483343095993

({'sensors': {'position': {'x': 5.2343035533096565,
    'y': -4.903382543795653,
    'z': 0.5498337806879715},
   'orientation': {'x': -0.09913444972125952,
    'y': 0.9860037340986153,
    'z': -0.1340484883243747,
    'w': 0.9071585393686104},
   'front_cam': None}},
 0,
 False,
 {})

shutdown request: new node registered with same name


Basically the enviroment needs to know two things: `World` and `Agent`. 

`KrockWebotsEnv` implements `WebotsEnv` that is a generic env for Webots

## Agent

The `Agent` class describes something that interacts to the enviroment. It provides a nice interface to adopt

### Usage

In [5]:
from agent import Agent

class DummyAgent(Agent):
    def act(self, env, *args, **kwargs):
        print('I am moving!')

a = DummyAgent()
a()

You need to first create the agent class and then call it to correctly initialise everything. You can think of it like a *lazy* loader.

### ROS
A `Rospy` agent is already implemented and can be found at `agent.RospyAgent`. It adds two methods to the basic `Agent`: `init_publishers` and `init_subscribers`. They must be overrided to initialise the subscribers and listeners


In [11]:
from agent import RospyAgent
from geometry_msgs.msg import PoseStamped
from sensor_msgs.msg import Joy

class MyRospyAgent(RospyAgent):
    def init_publishers(self):
        return {
            'joy': rospy.Publisher('/joy', Joy, queue_size=1),
        }

    def init_subscribers(self):
        return { 'pose': 
                rospy.Subscriber('/pose', PoseStamped, self.callback_pose)
               }

    def callback_pose(self, data):
        # this will triggers the `on_state_change` method in all callbacks
        self.state['pose'] = data

a = MyRospyAgent()
a()
print(a.subscribers)
print(a.publishers)

{'pose': <rospy.topics.Subscriber object at 0x7f25056d8828>}
{'joy': <rospy.topics.Publisher object at 0x7f25056d8dd8>}


### Callbacks
`Agent` implements the protocol `Callbackable` protocol. A callback can be
added by calling the `add_callback` or `add_callbacks`. 

The following interface expose the callback methods for `Agent`:

```python
class AgentCallback():
    def on_state_change(self, agent, key, value):
        pass

    def on_shut_down(self):
        pass
```

In [17]:
from agent.callbacks import AgentCallback

class MyCallback(AgentCallback):
    def on_state_change(self, agent, key, value):
        print(key, value)

Each `Agent` has a `state` that is just a `dict` . Every time it is changed it triggers the `on_state_change` event. This is usefull
to record stuff.

In [18]:
a = DummyAgent()
a.add_callback(MyCallback())
a()

a.state['foo'] = 'foo'

foo foo
